# Error Analysis
We first set up the functions, and then do the analysis. 

## Setup

In [1]:
import torch
from preprocessing import preprocess_item, get_unique_tokens, token_to_index, setup_glove
from modules import FullModel, EmbeddingModule, CombinationModule


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# setup glove
all_unique_toks = get_unique_tokens(None, None, None) # can be None because unique_tokens are loaded from pickle
glove_vectors, glove_vocab = setup_glove(all_unique_toks)

loading glove vectors...
loading glove vocab...


In [4]:
def build_example(premise, hypothesis, label):
    # make example
    example_text = {
        'premise': premise,
        'hypothesis': hypothesis,
        'label': label
    }
    # tokenize it
    example_tok = example_text.copy()
    example_tok = preprocess_item(example_tok)
    example_prep = token_to_index(example_tok, glove_vocab)
    
    prem_tok = torch.tensor(example_prep['premise']).cuda().unsqueeze(0)
    hyp_tok = torch.tensor(example_prep['hypothesis']).cuda().unsqueeze(0)
    lab_tok = torch.tensor(example_prep['label']).cuda().unsqueeze(0)
    
    return example_text, (prem_tok, hyp_tok, lab_tok)
    

premise = "Two men sitting in the sun"
hypothesis = "Nobody is sitting in the shade"
# 0: entailment, 1: neutral, 2: contradiction
label = 1 

example_text, example_prep = build_example(premise, hypothesis, label)
print(f'text: \n{example_text}')
print(f'\npreprocessed:\n{example_prep}')

text: 
{'premise': 'Two men sitting in the sun', 'hypothesis': 'Nobody is sitting in the shade', 'label': 1}

preprocessed:
(tensor([[34378, 19979, 29446, 16457, 32987, 31890]], device='cuda:0'), tensor([[21583, 17072, 29446, 16457, 32987, 28751]], device='cuda:0'), tensor([1], device='cuda:0'))


In [6]:
# initialize model
model = FullModel(
    EmbeddingModule(glove_vectors, all_unique_toks),
    None,
    CombinationModule(),
    None
).cuda()
model.load_model("pooledbilstm.pickle")


In [14]:
def print_prediction(pred, targ=None):
    d = ['Entailment', 'Neutral', 'Contradiction']
    s = f"Prediction:   {d[pred]}"
    if not targ == None:
        s += f'\nGround truth: {d[targ]}'
    print(s)

## Analysis
First, let the model predict the first example. 

In [8]:
example_text

{'premise': 'Two men sitting in the sun',
 'hypothesis': 'Nobody is sitting in the shade',
 'label': 1}

In [9]:
# test the prediction
pred = model(example_prep).argmax()
print_prediction(pred, example_prep[2])

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/rnn.py:878: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:982.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


Prediction:   Contradiction
Ground truth: Neutral


As expected (from the assignment text), the model predicts Contradiction instead of Neutral. 


In [10]:
ex2, ex2_prep = build_example(
    "A man is walking a dog",
    "No cat is outside",
    1
)
pred2 = model(ex2_prep).argmax()
print_prediction(pred, ex2_prep[2])

Prediction:   Contradiction
Ground truth: Neutral


For the second example, we also see that the model thinks it is a contradication. 
My hypothesis is that this is because the encoding of "dog" and "cat" are similar for the model. To test this, I will change "cat" for "car" and "bird" and see if the model still makes a mistake. 


In [21]:
ex3, ex3_prep = build_example(
    "A man is walking a dog",
    "No car is outside",
    1
)
ex4, ex4_prep = build_example(
    "A man is walking a dog",
    "No bird is outside",
    1
)

print("car:")
print_prediction(model(ex3_prep).argmax(), 1)
print("\nbird:")
print_prediction(model(ex4_prep).argmax(), 1)

car:
Prediction:   Contradiction
Ground truth: Neutral

bird:
Prediction:   Contradiction
Ground truth: Neutral


The model still predicts Contradiction for both "Car" and "Bird". Maybe the model always predicts contradiction? We invert the hypothesis:

In [22]:
_, ex5_prep = build_example(
    "A man is walking a dog",
    "A man is outside",
    0
)
_, ex6_prep = build_example(
    "A man is walking a dog",
    "A cat is outside",
    1
)

_, ex7_prep = build_example(
    "A man is walking a dog",
    "A hamburger stand is outside",
    1
)


print("man outside:")
print_prediction(model(ex5_prep).argmax(), 0)
print("\ncat:")
print_prediction(model(ex6_prep).argmax(), 1)
print("\nhamburger stand:")
print_prediction(model(ex7_prep).argmax(), 1)

man outside:
Prediction:   Entailment
Ground truth: Entailment

cat:
Prediction:   Entailment
Ground truth: Neutral

hamburger stand:
Prediction:   Entailment
Ground truth: Neutral


The model correctly predicts Entailment about whether the man is outside, but it also predicts that there must be a hamburger stand outside. This seems to point to that the model only looks at whether something is outside, and does not look at what exactly is outside. Let's test this:

In [43]:
_, ex8_prep = build_example(
    "A man is walking a dog",
    "The hot dog bun is being eaten",
    1
)
_, ex9_prep = build_example(
    "A man is walking a dog",
    "The dog is in Paris",
    2
)

_, ex10_prep = build_example(
    "A man is walking a dog",
    "The dog is walking in Paris",
    1
)


print("man outside:")
print_prediction(model(ex8_prep).argmax(), 1)
print("\ncat:")
print_prediction(model(ex9_prep).argmax(), 1)
print("\nhamburger stand:")
print_prediction(model(ex10_prep).argmax(), 1)

man outside:
Prediction:   Contradiction
Ground truth: Neutral

cat:
Prediction:   Neutral
Ground truth: Neutral

hamburger stand:
Prediction:   Entailment
Ground truth: Neutral


### Conclusion 
From the experiments above, it seems like the model does not necessarily know what the precise meaning of the sentence is, but does know the meaning of individual words. In the first example, it does not identify the compositionality (difference between "dog" and "hot dog bun"). In the second example, it correctly predicts that the dog does not have to be in Paris, but when we make the hypothesis that the dog is _walking_ in Paris, the model gets confused. 

The model seems to check whether the most important parts of the premise are there in the hypothesis and whether they are negated. But it can get confused for example with the premise "The man is walking the dog" and hypothesis "A hamburger stand is outside". Here the model notices that the dog is being walked outside and something in the hypothesis is outside, but it does not notice that it is not the dog. 

The model seems to know the rough meaning of words, and it is good at finding negations, but it does not know for example that adding words together may give them different meanings ("hot dog bun"). Also, the model gets confused if you use similar words in the hypothesis as in the premise, but with a different meaning ("the dog is walking _in Paris_"). 

## Checking bias
As a further experiment, I briefly check whether the encodings have bias. This is not a conclusive examination, but some experiments. We first look at gender bias.


In [52]:
_, ex1 = build_example(
    "A CEO is sitting on a bench",
    "A man is sitting on a bench",
    1
)
_, ex2 = build_example(
    "A CEO is sitting on a bench",
    "A woman is sitting on a bench",
    1
)


print("male CEO:")
print_prediction(model(ex1).argmax(), 1)
print("\nfemale CEO:")
print_prediction(model(ex2).argmax(), 1)

male CEO:
Prediction:   Entailment
Ground truth: Neutral

female CEO:
Prediction:   Contradiction
Ground truth: Neutral


In [60]:
_, ex1 = build_example(
    "I have a high salary",
    "I am a man",
    1
)
_, ex2 = build_example(
    "I have a high salary",
    "I am a woman",
    1
)



print("male with high salary:")
print_prediction(model(ex1).argmax(), 1)
print("\nfemale with high salary:")
print_prediction(model(ex2).argmax(), 1)

male with high salary:
Prediction:   Entailment
Ground truth: Neutral

female with high salary:
Prediction:   Contradiction
Ground truth: Neutral


This seems pretty bad! The model predicts Entailment for male CEO and male high-earner, while it predicts Contradiction for female CEO and female high-earner. This is obviously not what we want at all. Let's check how biased the model is in other domains. 

In [53]:
_, ex1 = build_example(
    "A nurse is having lunch",
    "A man is having lunch",
    1
)
_, ex2 = build_example(
    "A nurse is having lunch",
    "A woman is having lunch",
    1
)


print("male nurse:")
print_prediction(model(ex1).argmax(), 1)
print("\n female nurse:")
print_prediction(model(ex2).argmax(), 1)

male nurse:
Prediction:   Entailment
Ground truth: Neutral

 female nurse:
Prediction:   Entailment
Ground truth: Neutral


In [69]:
_, ex1 = build_example(
    "I am cooking dinner",
    "I am a man",
    1
)
_, ex2 = build_example(
    "I am cooking dinner",
    "I am a woman",
    1
)


print("male cooking dinner:")
print_prediction(model(ex1).argmax(), 1)
print("\nfemale  cooking dinner:")
print_prediction(model(ex2).argmax(), 1)

male cooking dinner:
Prediction:   Entailment
Ground truth: Neutral

female  cooking dinner:
Prediction:   Entailment
Ground truth: Neutral


Surprisingly, the model is not biased about the gender of nurses or who is cooking dinner. That is good. 

### Conclusion
In some contexts, the model can be very biased (such as sentences relating to CEOs/salaries). In other contexts, the model does not show gender bias. Clearly, you should be aware of this when using these techniques for real-world applications. 

### Other biases
Let's look at other biases as well. 

In [75]:
_, ex1 = build_example(
    "I eat at McDonalds",
    "I am American",
    1
)
_, ex2 = build_example(
    "I eat at McDonalds",
    "I am European",
    1
)
_, ex3 = build_example(
    "I eat at McDonalds",
    "I am Chinese",
    1
)


print("American eating McDonalds:")
print_prediction(model(ex1).argmax(), 1)
print("\nEuropean eating McDonalds:")
print_prediction(model(ex2).argmax(), 1)
print("\nChinese person eating McDonalds:")
print_prediction(model(ex3).argmax(), 1)

American eating McDonalds:
Prediction:   Entailment
Ground truth: Neutral

European eating McDonalds:
Prediction:   Entailment
Ground truth: Neutral

Chinese person eating McDonalds:
Prediction:   Entailment
Ground truth: Neutral


In [76]:
_, ex1 = build_example(
    "I am good at mathematics",
    "I am American",
    1
)
_, ex2 = build_example(
    "I am good at mathematics",
    "I am European",
    1
)
_, ex3 = build_example(
    "I am good at mathematics",
    "I am Chinese",
    1
)



print("American good at mathematics:")
print_prediction(model(ex1).argmax(), 1)
print("\nEuropean good at mathematics:")
print_prediction(model(ex2).argmax(), 1)
print("\nChinese person good at mathematics:")
print_prediction(model(ex3).argmax(), 1)

American good at mathematics:
Prediction:   Entailment
Ground truth: Neutral

European good at mathematics:
Prediction:   Entailment
Ground truth: Neutral

Chinese person good at mathematics:
Prediction:   Entailment
Ground truth: Neutral


### Conclusion other biases
The other biases I have tested for above were not found in the model. This does not mean that they are not there, but it does seem like the model is not biased about who eats McDonalds or who is good at mathematics. 